In [1]:
import numpy as np 
import pandas as pd 

from sklearn.feature_extraction.text import CountVectorizer
from keras.layers import Dropout
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential,clone_model
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras import optimizers
from keras.callbacks import ModelCheckpoint
import tensorflow as tf

import keras.backend as K

Using TensorFlow backend.


In [2]:
def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

In [3]:
from utils import *
df = functions.parse_file(r"raw_data/EmoContext/train.txt", "EmoContext")
df.head(5)

,id,turn1,turn2,turn3,label
0,0,Don't worry I'm girl,hmm how do I know if you are,What's ur name?,others
1,1,When did I?,saw many times i think -_-,No. I never saw you,angry
2,2,By,by Google Chrome,Where you live,others
3,3,U r ridiculous,I might be ridiculous but I am telling the truth.,U little disgusting whore,angry
4,4,Just for time pass,wt do u do 4 a living then,Maybe,others


In [4]:
text_data = []
for idx,row in df.iterrows():
    text_data.append("{}. {}. {}.".format(row['turn1'], row['turn2'], row['turn3']))

In [5]:
NR_WORDS = 5000
MAX_PROP_LENGTH = 200
tokenizer = Tokenizer(num_words=NR_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                   lower=True,split=' ')

tokenizer.fit_on_texts(text_data)

X_train = tokenizer.texts_to_sequences(text_data)
X_train = pad_sequences(X_train, maxlen = MAX_PROP_LENGTH)

In [6]:
def one_hot_vector(word,label=None):
    words = {"others": 0, "angry": 1, "sad":2, "happy": 3}
    if label == None:
        y = [0,0,0,0]
        y[words[word]] = 1
        return y
    if label == word:
        return [1,0]
    return [0,1]

Y_train_others = []
Y_train_angry = []
Y_train_sad = []
Y_train_happy = []

for idx,row in df.iterrows():
    Y_train_others.append(one_hot_vector(row['label'],"others"))
    Y_train_angry.append(one_hot_vector(row['label'],"angry"))
    Y_train_sad.append(one_hot_vector(row['label'],"sad"))
    Y_train_happy.append(one_hot_vector(row['label'],"happy"))

Y_train_others = np.array(Y_train_others)
Y_train_angry = np.array(Y_train_angry)
Y_train_sad = np.array(Y_train_sad)
Y_train_happy = np.array(Y_train_happy)

In [7]:
df = functions.parse_file(r"raw_data/EmoContext/devwithlabels.txt", "EmoContext")
df.head(5)

,id,turn1,turn2,turn3,label
0,0,Then dont ask me,YOURE A GUY NOT AS IF YOU WOULD UNDERSTAND,IM NOT A GUY FUCK OFF,angry
1,1,Mixed things such as??,the things you do.,Have you seen minions??,others
2,2,Today I'm very happy,and I'm happy for you ❤,I will be marry,happy
3,3,Woah bring me some,left it there oops,Brb,others
4,4,it is thooooo,I said soon master.,he is pressuring me,others


In [8]:
text_data = []
for idx,row in df.iterrows():
    text_data.append("{}. {}. {}.".format(row['turn1'], row['turn2'], row['turn3']))

In [9]:
tokenizer = Tokenizer(num_words=NR_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                   lower=True,split=' ')

tokenizer.fit_on_texts(text_data)

X_test = tokenizer.texts_to_sequences(text_data)
X_test = pad_sequences(X_test, maxlen = MAX_PROP_LENGTH)

In [10]:
def one_hot_vector(word,label=None):
    words = {"others": 0, "angry": 1, "sad":2, "happy": 3}
    if label == None:
        y = [0,0,0,0]
        y[words[word]] = 1
        return y
    if label == word:
        return [1,0]
    return [0,1]

Y_test_others = []
Y_test_angry = []
Y_test_sad = []
Y_test_happy = []

for idx,row in df.iterrows():
    Y_test_others.append(one_hot_vector(row['label'],"others"))
    Y_test_angry.append(one_hot_vector(row['label'],"angry"))
    Y_test_sad.append(one_hot_vector(row['label'],"sad"))
    Y_test_happy.append(one_hot_vector(row['label'],"happy"))

Y_test_others = np.array(Y_test_others)
Y_test_angry = np.array(Y_test_angry)
Y_test_sad = np.array(Y_test_sad)
Y_test_happy = np.array(Y_test_happy)

In [11]:
embed_dim = 128
lstm_out = 32
batch_size = 64

adam = optimizers.Adam(lr=0.01)
rmsprop = optimizers.RMSprop(lr=0.005, rho=0.9, epsilon=None, decay=0.0)


model = Sequential()
model.add(Embedding(NR_WORDS, embed_dim,input_length = X_train.shape[1]))
model.add(LSTM(lstm_out,dropout=0.5))
model.add(Dense(2,activation='softmax'))

In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 200, 128)          640000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                20608     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 66        
Total params: 660,674
Trainable params: 660,674
Non-trainable params: 0
_________________________________________________________________


In [13]:
mdcheck = ModelCheckpoint("trained_models/best_model_val_acc{val_acc:.4f}.h5", monitor='val_f1', save_best_only=True)

In [14]:
Y_train_angry = np.array(Y_train_angry)

model_angry = clone_model(model)
model_angry.compile(loss = 'binary_crossentropy', optimizer=rmsprop, metrics = ['accuracy', f1])
history = model_angry.fit(X_train, Y_train_angry,
                    validation_data=(X_test, Y_test_angry),
                    epochs=5, verbose=1, batch_size=batch_size,callbacks=[mdcheck])

Train on 30160 samples, validate on 2755 samples
Epoch 1/5
30160/30160 [==============================] - 69s 2ms/step - loss: 0.1947 - acc: 0.9297 - f1: 0.8670 - val_loss: 0.5677 - val_acc: 0.7891 - val_f1: 0.4789
Epoch 2/5
30160/30160 [==============================] - 67s 2ms/step - loss: 0.1478 - acc: 0.9469 - f1: 0.9069 - val_loss: 0.5565 - val_acc: 0.8087 - val_f1: 0.4881
Epoch 3/5
30160/30160 [==============================] - 67s 2ms/step - loss: 0.1342 - acc: 0.9521 - f1: 0.9159 - val_loss: 0.5016 - val_acc: 0.7953 - val_f1: 0.4825
Epoch 4/5
30160/30160 [==============================] - 68s 2ms/step - loss: 0.1265 - acc: 0.9544 - f1: 0.9194 - val_loss: 0.5352 - val_acc: 0.8145 - val_f1: 0.4819
Epoch 5/5
30160/30160 [==============================] - 68s 2ms/step - loss: 0.1189 - acc: 0.9578 - f1: 0.9261 - val_loss: 0.5655 - val_acc: 0.8167 - val_f1: 0.4815


In [15]:
Y_train_sad = np.array(Y_train_sad)

model_sad = clone_model(model)
model_sad.compile(loss = 'binary_crossentropy', optimizer=rmsprop, metrics = ['accuracy', f1])
history = model_sad.fit(X_train, Y_train_sad,
                    validation_data=(X_test, Y_test_sad),
                    epochs=5, verbose=1, batch_size=batch_size,callbacks=[mdcheck])

Train on 30160 samples, validate on 2755 samples
Epoch 1/5
30160/30160 [==============================] - 69s 2ms/step - loss: 0.2202 - acc: 0.9180 - f1: 0.8424 - val_loss: 0.3437 - val_acc: 0.8730 - val_f1: 0.4930
Epoch 2/5
30160/30160 [==============================] - 69s 2ms/step - loss: 0.1613 - acc: 0.9442 - f1: 0.9007 - val_loss: 0.3697 - val_acc: 0.8697 - val_f1: 0.5007
Epoch 3/5
30160/30160 [==============================] - 68s 2ms/step - loss: 0.1439 - acc: 0.9512 - f1: 0.9128 - val_loss: 0.3561 - val_acc: 0.8849 - val_f1: 0.4999
Epoch 4/5
30160/30160 [==============================] - 68s 2ms/step - loss: 0.1316 - acc: 0.9562 - f1: 0.9211 - val_loss: 0.3656 - val_acc: 0.8711 - val_f1: 0.4948
Epoch 5/5
30160/30160 [==============================] - 72s 2ms/step - loss: 0.1230 - acc: 0.9591 - f1: 0.9263 - val_loss: 0.3668 - val_acc: 0.8802 - val_f1: 0.4976


In [16]:
Y_train_happy = np.array(Y_train_happy)

model_happy = clone_model(model)
model_happy.compile(loss = 'binary_crossentropy', optimizer=rmsprop, metrics = ['accuracy', f1])
history = model_happy.fit(X_train, Y_train_happy,
                    validation_data=(X_test, Y_test_happy),
                    epochs=5, verbose=1, batch_size=batch_size,callbacks=[mdcheck])

Train on 30160 samples, validate on 2755 samples
Epoch 1/5
30160/30160 [==============================] - 71s 2ms/step - loss: 0.1775 - acc: 0.9357 - f1: 0.8451 - val_loss: 0.3141 - val_acc: 0.8922 - val_f1: 0.5202
Epoch 2/5
30160/30160 [==============================] - 70s 2ms/step - loss: 0.1325 - acc: 0.9538 - f1: 0.8974 - val_loss: 0.3471 - val_acc: 0.8704 - val_f1: 0.5188
Epoch 3/5
30160/30160 [==============================] - 69s 2ms/step - loss: 0.1179 - acc: 0.9580 - f1: 0.9064 - val_loss: 0.2997 - val_acc: 0.9056 - val_f1: 0.5422
Epoch 4/5
30160/30160 [==============================] - 71s 2ms/step - loss: 0.1105 - acc: 0.9605 - f1: 0.9123 - val_loss: 0.3028 - val_acc: 0.9013 - val_f1: 0.5253
Epoch 5/5
30160/30160 [==============================] - 67s 2ms/step - loss: 0.1035 - acc: 0.9642 - f1: 0.9209 - val_loss: 0.3461 - val_acc: 0.8915 - val_f1: 0.5324


In [17]:
Y_train_others = np.array(Y_train_others)

model_others = clone_model(model)
model_others.compile(loss = 'binary_crossentropy', optimizer=rmsprop, metrics = ['accuracy', f1])
history = model_others.fit(X_train, Y_train_others,
                    validation_data=(X_test, Y_test_others),
                    epochs=5, verbose=1, batch_size=batch_size,callbacks=[mdcheck])

Train on 30160 samples, validate on 2755 samples
Epoch 1/5
30160/30160 [==============================] - 75s 2ms/step - loss: 0.3740 - acc: 0.8395 - f1: 0.8358 - val_loss: 0.9653 - val_acc: 0.5038 - val_f1: 0.4237
Epoch 2/5
30160/30160 [==============================] - 75s 2ms/step - loss: 0.3021 - acc: 0.8803 - f1: 0.8787 - val_loss: 1.1434 - val_acc: 0.5118 - val_f1: 0.4242
Epoch 3/5
30160/30160 [==============================] - 69s 2ms/step - loss: 0.2767 - acc: 0.8908 - f1: 0.8892 - val_loss: 1.0985 - val_acc: 0.5354 - val_f1: 0.4372
Epoch 4/5
30160/30160 [==============================] - 71s 2ms/step - loss: 0.2552 - acc: 0.9027 - f1: 0.9012 - val_loss: 1.3080 - val_acc: 0.5238 - val_f1: 0.4375
Epoch 5/5
30160/30160 [==============================] - 69s 2ms/step - loss: 0.2379 - acc: 0.9097 - f1: 0.9085 - val_loss: 1.0775 - val_acc: 0.5615 - val_f1: 0.4474


In [18]:
model_json = model.to_json()
with open("lstm_normal_model_others.json", "w") as outfile:
    outfile.write(model_json)
model.save_weights("lstm_normal_12_epochs_cv_others.h5")

In [19]:
model_json = model.to_json()
with open("lstm_normal_model_angry.json", "w") as outfile:
    outfile.write(model_json)
model.save_weights("lstm_normal_12_epochs_cv_angry.h5")

In [20]:
model_json = model.to_json()
with open("lstm_normal_model_sad.json", "w") as outfile:
    outfile.write(model_json)
model.save_weights("lstm_normal_12_epochs_cv_sad.h5")

In [21]:
model_json = model.to_json()
with open("lstm_normal_model_happy.json", "w") as outfile:
    outfile.write(model_json)
model.save_weights("lstm_normal_12_epochs_cv_happy.h5")